# Curvature calculation -- Python only

This notebook processes a collection of line vectors into curvature estimates

In [7]:
import os, sys

import numpy as np
from numpy import random

import rasterio
from rasterio import features
from rasterio import transform
from rasterio.transform import Affine

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import *
from shapely.ops import split, transform
from shapely.wkt import loads

sys.path.append('../../src/')

from gostrocks.src.GOSTRocks.misc import tPrint

### Setup

File paths

In [2]:
data_dir = r'../../data'

rd_dir = r'roads'
base_dir = r'rast_inputs'

speed_dir = r'speed_rasts'

Projections

In [3]:
# change this to whatever the desired metric output projection is -- this notebook assumes you want outputs in meters
dest_crs = 'EPSG:32642'

Speed column we're workign on

In [4]:
speed_column = 'dry_speed'

Functions

In [9]:
# from https://gis.stackexchange.com/questions/378806/multi-part-geometries-do-not-provide-a-coordinate-sequence-error-when-extracti

def add_start_end_nodes_to_gdf(gdf):
    
# This function adds start and end nodes to geodataframe

    gdf['start_node'] = None
    gdf['end_node'] = None

    for index, row in gdf.iterrows():
        coords = [(coords) for coords in list(row['geometry'].coords)]
        start_node, end_node = [coords[i] for i in (0, -1)]
        gdf.at[index, 'start_node'] = start_node
        gdf.at[index, 'end_node'] = end_node


In [10]:
# from https://gis.stackexchange.com/questions/297134/shapely-floating-problems-with-split/327287#327287

def get_linesegments(line, n):
    segments = [line]
    points = MultiPoint([line.interpolate(i/n, normalized=True) for i in range(1, n)])
    for point in points:
        lastline = segments[-1]
        for ix, (c1, c2) in enumerate(zip(lastline.coords[:-1], lastline.coords[1:])):
            if LineString([c1, c2]).distance(point) < 1e-8:
                segments[-1] = LineString(lastline.coords[:ix+1] + [point.coords[0]])
                if point.coords[0] == c2:
                    segments.append(LineString(lastline.coords[ix+1:]))
                else:
                    segments.append(LineString([point.coords[0]] + lastline.coords[ix+1:]))
                break
    return GeometryCollection(segments)

## Load in and process roads vectors

In [5]:
prop = gpd.read_file(os.path.join(data_dir,rd_dir,'Proposed/KP_RoadforUpgrade_210819.gpkg'),driver="GPKG").to_crs(dest_crs)

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [6]:
prop

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,SN,Road_Name,Details,District_Name,Road_Class,Pavement_type,Road_Actual_Length_km,Road _Proposed_Length_km,geometry
0,27247219,5112,trunk,Drosh-Chitral West Road,N-45,B,0,0,F,F,7,Drosh-Chitral West Road,None,Lower Chitral,Collector Road,Shingle,20.00,None,"MULTILINESTRING ((754400.310 3941840.318, 7544..."
1,27247263,5115,tertiary,Drosh-Chitral West Road,None,B,0,0,F,F,7,Drosh-Chitral West Road,None,Lower Chitral,Collector Road,Shingle,20.00,None,"MULTILINESTRING ((752244.146 3969417.479, 7522..."
2,27247271,5115,tertiary,Drosh-Chitral West Road (Ayun-Orghuch-Chitral ...,None,B,0,0,F,F,10,Drosh-Chitral West Road (Ayun-Orghuch-Chitral ...,None,Lower Chitral,Collector Road,Shingle,20.00,None,"MULTILINESTRING ((750655.660 3957561.958, 7506..."
3,27247955,5115,tertiary,Drosh-Chitral West Road,None,B,0,0,F,F,7,Drosh-Chitral West Road,None,Lower Chitral,Collector Road,Shingle,20.00,None,"MULTILINESTRING ((751512.661 3969798.506, 7515..."
4,27247956,5115,tertiary,Drosh-Chitral West Road,None,B,0,1,T,F,7,Drosh-Chitral West Road,None,Lower Chitral,Collector Road,Shingle,20.00,None,"MULTILINESTRING ((751963.584 3969457.886, 7518..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,None,0,unclassified,Shoghor-Karimabad Road,None,B,0,0,T,F,11,Shoghor-Karimabad Road,None,Lower Chitral,Village road,Shingle,30.00,None,"MULTILINESTRING ((749216.431 3989406.971, 7491..."
276,None,0,unclassified,Shoghor-Karimabad Road,None,B,0,0,T,F,11,Shoghor-Karimabad Road,None,Lower Chitral,Village road,Shingle,30.00,None,"MULTILINESTRING ((749155.992 3989697.717, 7491..."
277,968422788,5115,tertiary,Jamrud Bypass to Koi Sher Haider Baza,None,B,0,0,F,F,80,Jamrud Bypass to Koi Sher Haider Baza,None,Khuber,District Road,None,12.00,None,"MULTILINESTRING ((720632.661 3759432.366, 7205..."
278,311561900,0,primary,Mastuj Boroghol Pass Road,None,B,0,0,F,F,3,Mastuj Boroghol Pass Road,None,Upper Chitral,Collector Road,Shingle,None,None,"MULTILINESTRING ((815812.425 4020685.300, 8158..."


In [7]:
prop_slim = prop.drop_duplicates(['SN','Road_Class','Pavement_type']).drop(['oneway','bridge','layer','tunnel','maxspeed','geometry'],axis=1)

In [8]:
prop_slim.head()

,osm_id,code,fclass,name,ref,SN,Road_Name,Details,District_Name,Road_Class,Pavement_type,Road_Actual_Length_km,Road _Proposed_Length_km
0,27247219,5112,trunk,Drosh-Chitral West Road,N-45,7,Drosh-Chitral West Road,None,Lower Chitral,Collector Road,Shingle,20.00,None
2,27247271,5115,tertiary,Drosh-Chitral West Road (Ayun-Orghuch-Chitral ...,None,10,Drosh-Chitral West Road (Ayun-Orghuch-Chitral ...,None,Lower Chitral,Collector Road,Shingle,20.00,None
5,76559841,5115,tertiary,Jamrud Bypass to Koi Sher Haider Baza,None,80,Jamrud Bypass to Koi Sher Haider Baza,None,Khuber,District Road,None,12.00,None
7,194779126,5115,tertiary,Khar Qamar to Lataka Tehsil Datta Khel,None,86,Khar Qamar to Lataka Tehsil Datta Khel,"Improvement, widening and Reconstruction of BT...",Noerth Wazir,Access Road,Gravel,20.00,None
12,199252267,5121,unclassified,Shoghor-Karimabad Road,None,11,Shoghor-Karimabad Road,None,Lower Chitral,Village road,Shingle,30.00,None


## Create curvature

Actual data transformations

In [11]:
t = prop[['SN','geometry']]

In [12]:
t = gpd.GeoDataFrame(t.geometry.explode()).reset_index().rename({'level_0':'SN'},axis=1)

**calculate the point that lies at every 100m of a line**

Splitting lines every 100

In [178]:
seg1

SN                                                          0
level_1                                                     0
geometry    LINESTRING (754400.309967901 3941840.317854876...
Name: 0, dtype: object

In [180]:
seg1.SN

0

Now do it at scale

In [187]:
roads_100m_segs = []

for sn, lin in zip(t.SN, t.geometry):

    for i in get_linesegments(lin, (int(np.ceil(lin.length / 100)))):
        
        seg_100m = (sn,i)
        
        roads_100m_segs.append(seg_100m)
    


In [191]:
prop_100m_segs_gdf = gpd.GeoDataFrame(pd.DataFrame(roads_100m_segs,columns=['SN','geometry']))
add_start_end_nodes_to_gdf(prop_100m_segs_gdf)

In [193]:
prop_100m_segs_gdf.head()

,SN,geometry,start_node,end_node
0,0,"LINESTRING (754400.310 3941840.318, 754400.683...","(754400.309967901, 3941840.3178548757)","(754328.5327509238, 3941900.5902303555)"
1,0,"LINESTRING (754328.533 3941900.590, 754298.427...","(754328.5327509238, 3941900.5902303555)","(754231.3878951209, 3941908.5045265793)"
2,0,"LINESTRING (754231.388 3941908.505, 754208.595...","(754231.3878951209, 3941908.5045265793)","(754134.4021555999, 3941908.7992968)"
3,0,"LINESTRING (754134.402 3941908.799, 754129.628...","(754134.4021555999, 3941908.7992968)","(754131.3153452484, 3942001.6761530666)"
4,0,"LINESTRING (754131.315 3942001.676, 754131.561...","(754131.3153452484, 3942001.6761530666)","(754136.5124096291, 3942101.0574579383)"


### Calculate actual curvature

In [ ]:
# # curvature calculation: https://gis.stackexchange.com/questions/101727/radius-of-curvature-for-curves-in-road-segments-for-entire-road-network
# k = change in (np.tan(angle) * length)

In [218]:
# test examlpe 
prop_100m_segs_gdf.iloc[0].geometry.length / \
np.sqrt(((prop_100m_segs_gdf.iloc[0].start_node[0] - prop_100m_segs_gdf.iloc[0].end_node[0]) ** 2) + \
        ((prop_100m_segs_gdf.iloc[0].start_node[1] - prop_100m_segs_gdf.iloc[0].end_node[1]) ** 2))

1.0627991554143381

In [235]:
# iterate over each row and calculate its sinuosity. Then use the list of values to calculate an additional data column on the main GDF
sin_list = []

for idx, seg_row in prop_100m_segs_gdf.iterrows():
    sinuosity = seg_row.geometry.length / \
                            np.sqrt(((seg_row.start_node[0] - seg_row.end_node[0]) ** 2) + \
                                    ((seg_row.start_node[1] - seg_row.end_node[1]) ** 2))
    
    sin_list.append(sinuosity)

prop_100m_segs_gdf['sinuosity'] = pd.Series(sin_list)

In [238]:
# of value interpreting the results: https://gis.stackexchange.com/questions/202439/calculating-sinuosity-of-each-road-segment

prop_100m_segs_gdf

,SN,geometry,start_node,end_node,sinuosity
0,0,"LINESTRING (754400.310 3941840.318, 754400.683...","(754400.309967901, 3941840.3178548757)","(754328.5327509238, 3941900.5902303555)",1.062799
1,0,"LINESTRING (754328.533 3941900.590, 754298.427...","(754328.5327509238, 3941900.5902303555)","(754231.3878951209, 3941908.5045265793)",1.022019
2,0,"LINESTRING (754231.388 3941908.505, 754208.595...","(754231.3878951209, 3941908.5045265793)","(754134.4021555999, 3941908.7992968)",1.027083
3,0,"LINESTRING (754134.402 3941908.799, 754129.628...","(754134.4021555999, 3941908.7992968)","(754131.3153452484, 3942001.6761530666)",1.071934
4,0,"LINESTRING (754131.315 3942001.676, 754131.561...","(754131.3153452484, 3942001.6761530666)","(754136.5124096291, 3942101.0574579383)",1.000962
...,...,...,...,...,...
9839,279,"LINESTRING (819527.794 4024630.652, 819521.615...","(819527.793785164, 4024630.651793612)","(819454.0708781193, 4024564.7423587353)",1.009040
9840,279,"LINESTRING (819454.071 4024564.742, 819449.357...","(819454.0708781193, 4024564.7423587353)","(819361.1786023861, 4024528.83952557)",1.001951
9841,279,"LINESTRING (819361.179 4024528.840, 819337.182...","(819361.1786023861, 4024528.83952557)","(819272.194563076, 4024484.8908234197)",1.005421
9842,279,"LINESTRING (819272.195 4024484.891, 819258.141...","(819272.194563076, 4024484.8908234197)","(819202.8857038105, 4024416.83933828)",1.027291


In [247]:
# can't export tuples
prop_100m_segs_gdf.start_node = prop_100m_segs_gdf.start_node.astype(str)
prop_100m_segs_gdf.end_node = prop_100m_segs_gdf.end_node.astype(str)

In [249]:
# condense to original proposed roads
prop_d = prop_100m_segs_gdf.dissolve(by='SN',aggfunc='mean').reset_index().set_crs(dest_crs)

In [250]:
np.max(prop_d.sinuosity)

1.625102041331965

#### Export

In [254]:
prop_d.to_file(os.path.join(data_dir,'vect_out/Sinuosity_test.gpkg'),driver="GPKG")
prop_d.to_file(os.path.join(data_dir,'vect_out/Sinuosity_test.shp'))

## All roads version

In [5]:
master_rds = gpd.read_file(os.path.join(data_dir,rd_dir,'combined_KPK_mnt_splt_utm.shp')).to_crs(dest_crs)

In [11]:
add_start_end_nodes_to_gdf(master_rds)

In [13]:
# iterate over each row and calculate its sinuosity. Then use the list of values to calculate an additional data column on the main GDF
sin_list = []

for idx, seg_row in master_rds.iterrows():
    sinuosity = seg_row.geometry.length / \
                            np.sqrt(((seg_row.start_node[0] - seg_row.end_node[0]) ** 2) + \
                                    ((seg_row.start_node[1] - seg_row.end_node[1]) ** 2))
    
    sin_list.append(sinuosity)

master_rds['sinuosity'] = pd.Series(sin_list)

<ipython-input-13-290289f1f34e>:5: RuntimeWarning: divide by zero encountered in double_scalars
  sinuosity = seg_row.geometry.length / \


In [16]:
# old

master['Elev_Terrain'] = pd.cut(master['Z_Mean'], [-np.inf, 1500, 2499, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here

# new
# I adjusted these numbers down basked on a desk review -- Robert

master['Slope_Terrain'] = pd.cut(master['Avg_Slope'], [-np.inf, 8, 16, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here

Index(['fid_1', 'cat', 'fid_12', 'Road_ID', 'Road_No', 'Road_Name', 'SegNo',
       'Seg_Name', 'Road_Class', 'Pavement_T', 'Avg_Right', 'BT_Width',
       'Shoulder_T', 'Shoulder_W', 'Road_Lengt', 'No_of_Brid', 'No_of_Culv',
       'TYPE', 'Year_of_Co', 'Contractor', 'Executing', 'Remarks', 'Name',
       'Division', 'SHAPE_Leng', 'District', 'ADP', 'ADP_Serial', 'Sub_Scheme',
       'Search_Nam', 'Search_Num', 'road', 'x_cor', 'y_Cor', 'ID',
       'ROADS_Join', 'MNT_Num', 'MNT_Distri', 'MNT_S_No', 'MNT_Road_N',
       'MNT_Road_1', 'MNT_Road_C', 'MNT_Road_2', 'MNT_Road_l', 'MNT_Paveme',
       'MNT_Year_o', 'MNT_RONET_', 'MNT_Traffi', 'MNT_Carria', 'MNT_Number',
       'MNT_Seg_Na', 'MNT_RONE_1', 'MNT_RONE_2', 'MNT_Functy', 'MNT_Traf_1',
       'MNT_Surfac', 'MNT_Condit', 'MNT_Final_', 'Z_Mean', 'Avg_Slope',
       'geometry', 'start_node', 'end_node', 'sinuosity'],
      dtype='object')

In [17]:
master_rds['Length'] = master_rds.geometry.length

In [27]:
# Elevation

master_rds['Elev_Terrain'] = pd.cut(master_rds['Z_Mean'], [-np.inf, 1500, 2499, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']).astype(str) # change labels here

# Slope

master_rds['Slope_Terrain'] = pd.cut(master_rds['Avg_Slope'], [-np.inf, 8, 16, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']).astype(str) # change labels herelabels = ['Plains', 'Hills', 'Mountains']) # change labels here

# Curvature

master_rds['Curvature_Terrain'] = pd.cut(master_rds['sinuosity'], [-np.inf, 1.1, 1.3, np.inf], 
                           labels = ['Plains', 'Hills', 'Mountains']).astype(str) # change labels here

In [29]:
master_rds.drop(['start_node','end_node'],axis=1).to_file(os.path.join(data_dir,rd_dir,'curvature_terrain_test.gpkg'),driver="GPKG")